In [ ]:
import yfinance as yf
import alpaca_trade_api as tradeapi
import time
from datetime import datetime, timedelta

# Your Alpaca API key and secret
alpaca_api_key = ""
alpaca_secret_key = "it8zx7g8jbeBoqwwJjUXYxH9NyKVQuoXoZooMY7S"

# Initialize the Alpaca API
api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, base_url='https://paper-api.alpaca.markets')

# Define the trading symbol
symbol = "BTC-USD"  # replace with your chosen symbol

# Define your portfolio
portfolio_value = 100000  # starting portfolio value
btc_holdings = 0  # starting bitcoin holdings
trade_size = 0.1  # trade size as a fraction of portfolio value
stop_loss = 0.02  # stop loss threshold as a fraction of buy price
take_profit = 0.03  # take profit threshold as a fraction of buy price
in_position = False  # tracks whether you're currently holding the asset
buy_price = None  # stores the price at which the asset was bought

# Define a time span for which to fetch historical data
time_span = timedelta(minutes=5)

# Insert your candlestick pattern functions here
def is_bullish_engulfing(df):
    row1, row2 = df.iloc[-2], df.iloc[-1]
    if row1['Close'] < row1['Open'] and row2['Close'] > row2['Open'] and row2['Open'] < row1['Close'] and row2['Close'] > row1['Open']:
        return True
    return False
    
def is_morning_star(df):
    row1, row2, row3 = df.iloc[-3], df.iloc[-2], df.iloc[-1]
    if row1['Close'] < row1['Open'] and abs(row2['Close'] - row2['Open']) <= 0.01 and row3['Close'] > row3['Open'] and row3['Close'] >= 0.95 * row1['Open']:
        return True
    return False
    
def is_bullish_key_reversal(df):
    row1, row2 = df.iloc[-2], df.iloc[-1]
    if row2['Low'] < row1['Low'] and row2['Close'] > row2['Open'] and row2['Close'] > row1['Close']:
        return True
    return False

def is_three_white_soldiers(df):
    last_three_candles = df.iloc[-3:]
    if (last_three_candles['Open'] < last_three_candles['Close']).all() and \
       (last_three_candles['High'] == last_three_candles['Close']).all() and \
       (last_three_candles['Open'].shift(-1) > last_three_candles['Close'].shift(-2)).all():
        return True
    return False

def is_rising_three_methods(df):
    last_five_candles = df.iloc[-5:]
    first_candle = last_five_candles.iloc[0]
    last_candle = last_five_candles.iloc[-1]
    if first_candle['Open'] < first_candle['Close'] and last_candle['Open'] < last_candle['Close']:
        middle_three_candles = last_five_candles.iloc[1:4]
        if (middle_three_candles['Open'] > middle_three_candles['Close']).all() and \
           (middle_three_candles['High'] < first_candle['High']).all() and \
           (middle_three_candles['Low'] > first_candle['Low']).all():
            if last_candle['Close'] > first_candle['Close']:
                return True
    return False

def is_hammer(df):
    row = df.iloc[-1]
    if (row['Open'] < row['Close']) and ((row['High'] - row['Low']) > 3 * (row['Close'] - row['Open'])) and (row['Close'] - row['Low'] < 0.2 * (row['High'] - row['Low'])):
        # Check for a downward trend over the previous 5 periods
        if df['Close'].iloc[-5] > df['Open'].iloc[-5]:
            return True
    return False

def is_bearish_engulfing(df):
    row1, row2 = df.iloc[-2], df.iloc[-1]
    if row1['Close'] > row1['Open'] and row2['Close'] < row2['Open'] and row2['Open'] > row1['Close'] and row2['Close'] < row1['Open']:
        return True
    return False

def is_evening_star(df):
    row1, row2, row3 = df.iloc[-3], df.iloc[-2], df.iloc[-1]
    if row1['Close'] > row1['Open'] and abs(row2['Close'] - row2['Open']) <= 0.01 and row3['Close'] < row3['Open'] and row3['Close'] <= 1.05 * row1['Open']:
        return True
    return False

def is_bearish_key_reversal(df):
    row1, row2 = df.iloc[-2], df.iloc[-1]
    if row2['High'] > row1['High'] and row2['Close'] < row2['Open'] and row2['Close'] < row1['Close']:
        return True
    return False

def is_three_black_crows(df):
    c1 = df['Close'].shift(1) > df['Open'].shift(1)
    c2 = df['Open'].shift(1) > df['Open'].shift(2)
    c3 = df['Close'].shift(2) > df['Open'].shift(2)
    c4 = df['Open'].shift(2) > df['Open'].shift(3)
    c5 = df['Close'].shift(3) > df['Open'].shift(3)
    return (c1 & c2 & c3 & c4 & c5).any()

def is_falling_three_methods(df):
    c1 = df['Close'].shift(4) > df['Open'].shift(4)
    c2 = df['Close'].shift(3) < df['Open'].shift(3)
    c3 = df['Close'].shift(2) < df['Open'].shift(2)
    c4 = df['Close'].shift(1) < df['Open'].shift(1)
    c5 = df['Close'] < df['Open']
    cond1 = df['Close'].shift(3) > df['Close'].shift(4)
    cond2 = df['Close'].shift(2) > df['Close'].shift(4)
    cond3 = df['Close'].shift(1) > df['Close'].shift(4)
    cond4 = df['Close'] < df['Close'].shift(4)
    return (c1 & c2 & c3 & c4 & c5 & cond1 & cond2 & cond3 & cond4).any()


while True:
    # Get the most recent data from Yahoo Finance
    end = datetime.now()
    start = end - time_span
    data = yf.download(symbol, start=start, end=end, interval='1m')

    row = data.iloc[-1]
    prev_rows = data.iloc[-6:-1]

    if not in_position:
        if is_bullish_engulfing(prev_rows) or is_morning_star(prev_rows) or is_bullish_key_reversal(prev_rows) or \
                is_three_white_soldiers(prev_rows) or is_rising_three_methods(prev_rows) or is_hammer(prev_rows):
            num_units_to_buy = (portfolio_value * trade_size) / row['Close']
            btc_holdings += num_units_to_buy
            portfolio_value -= num_units_to_buy * row['Close']

            in_position = True
            buy_price = row['Close']  # store the buy price
            print(f"Buy {num_units_to_buy} units at ${buy_price} each on {row.name}")
            api.submit_order(
                symbol=symbol,
                qty=num_units_to_buy,
                side='buy',
                type='market',
                time_in_force='gtc'
            )
    else:
        sell_condition = is_bearish_engulfing(prev_rows) or is_evening_star(prev_rows) or is_bearish_key_reversal(
            prev_rows) or is_three_black_crows(prev_rows) or is_falling_three_methods(prev_rows)
        stop_loss_condition = row['Close'] <= buy_price * (1 - stop_loss)
        take_profit_condition = row['Close'] >= buy_price * (1 + take_profit)

        if sell_condition or stop_loss_condition or take_profit_condition:
            num_units_to_sell = btc_holdings * trade_size
            btc_holdings -= num_units_to_sell
            portfolio_value += num_units_to_sell * row['Close']

            in_position = False
            print(f"Sell {num_units_to_sell} units at ${row['Close']} each on {row.name}")
            api.submit_order(
                symbol=symbol,
                qty=num_units_to_sell,
                side='sell',
                type='market',
                time_in_force='gtc'
            )

    time.sleep(60)  # wait for 60 seconds before the next iteration


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
